In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
changeupDL = pd.read_csv('DraftLeagueChangeup.csv')
changeupMLB = pd.read_csv('MLBchangeup.csv')

In [3]:
changeupDL = changeupDL.astype(str)
changeupMLB = changeupMLB.astype(str)

In [4]:
changeupDL['league'] = 'Draft League'
changeupMLB['league'] = 'MLB'

In [5]:
pitchersDL = list(set(changeupDL['pitcher']))

In [6]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [7]:
def get_index_from_pitcher(pitcher):
    return changeup[changeup.pitcher == pitcher].index[0]

In [8]:
def get_pitcher_from_index(index):
    return changeup[changeup.index == index].values[0][0]

In [9]:
for x in pitchersDL:
    
    try:

        changeup = pd.concat([changeupDL, changeupMLB])

        changeup["combined_features"] = changeup.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(changeup["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        changeup = changeup[(changeup['pitcher'] == pitcher_like) | (changeup['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)

        i=0

        for pitcher in sorted_similar_pitchers:
            print(get_pitcher_from_index(pitcher[0]),", Cosine Sim: ", sorted_similar_pitchers[i][1])
            i=i+1
            if i>1:
                break
                
    except IndexError:
        print(x, ', No Similarity')
        pass

Goodwin, Sean , Cosine Sim:  1.0000000000000002
Duffy, Matt , Cosine Sim:  0.2721655269759087
Becker, Austin , Cosine Sim:  0.9999999999999999
Becker, Austin , No Similarity
Ottinger, Zach , Cosine Sim:  1.0000000000000002
Demeritte, Travis , Cosine Sim:  0.30151134457776363
Fernandez, Ronaldo , Cosine Sim:  0.9999999999999999
Schrock, Max , Cosine Sim:  0.31622776601683794
Bosch, Carter , Cosine Sim:  1.0
Barlow, Joe , Cosine Sim:  0.38138503569823695
Godwin, Connor , Cosine Sim:  1.0000000000000002
Herrera, Kelvin , Cosine Sim:  0.3333333333333333
Blatter, Brock , Cosine Sim:  1.0
Barlow, Joe , Cosine Sim:  0.38138503569823695
Figueroa, Ryan , Cosine Sim:  0.9999999999999999
Burdi, Nick , Cosine Sim:  0.3585685828003181
Stewart, Justin , Cosine Sim:  0.9999999999999999
Gutierrez, Vladimir , Cosine Sim:  0.33541019662496846
Sikorski, Easton , Cosine Sim:  0.9999999999999999
Newcomb, Sean , Cosine Sim:  0.31622776601683794
Alba, Max , Cosine Sim:  1.0000000000000002
Walker, Neil , Cosi

Elissalt, Frank , Cosine Sim:  0.9999999999999999
Hembree, Heath , Cosine Sim:  0.31622776601683794
Diedrich, Sayer , Cosine Sim:  0.9999999999999999
Schrock, Max , Cosine Sim:  0.31622776601683794
Fenlong, Connor , Cosine Sim:  0.9999999999999999
Fenlong, Connor , No Similarity
Cheeley, Austin , Cosine Sim:  0.9999999999999999
Pagán, Emilio , Cosine Sim:  0.3651483716701108
Gilchrist, Malcolm , Cosine Sim:  1.0000000000000002
Rogers, Taylor , Cosine Sim:  0.31622776601683794
Walling, Andrew , Cosine Sim:  1.0000000000000002
Schrock, Max , Cosine Sim:  0.3333333333333333
Suero, Jose , Cosine Sim:  1.0000000000000002
Kay, Anthony , Cosine Sim:  0.3333333333333333
Schomberg, Will , Cosine Sim:  0.9999999999999999
Schomberg, Will , No Similarity
Escano, Antonio , Cosine Sim:  0.9999999999999999
Keller, Mitch , Cosine Sim:  0.33806170189140666
Metcalf, Joseph , Cosine Sim:  0.9999999999999999
Mayers, Mike , Cosine Sim:  0.3872983346207417
Bucknam, Micah , Cosine Sim:  0.9999999999999999
Bu